Kears NN and tensorboard v1

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import FalseNegatives
from tensorflow.keras.metrics import FalsePositives
from tensorflow.keras.metrics import TrueNegatives
from tensorflow.keras.metrics import TruePositives
from tensorflow.keras.metrics import PrecisionAtRecall
from tensorflow.keras.metrics import RecallAtPrecision
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.metrics import MeanSquaredLogarithmicError
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
import re

In [2]:
#path to personal environment
path = '/content/drive/MyDrive/GHG/MSDS696/MSDS696_GHG/'


In [3]:
df = pd.read_csv(path+'Data/GHG_Post_Outlier.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44079 entries, 0 to 44078
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   account_name                44079 non-null  object 
 1   account_id                  44079 non-null  int64  
 2   Year                        44079 non-null  int64  
 3   incorporated_country        44079 non-null  object 
 4   Primary activity            44079 non-null  object 
 5   Primary sector              44079 non-null  object 
 6   Market_Cap_USD              37591 non-null  float64
 7   Third_party_verification    44055 non-null  object 
 8   Revenue_USD                 37799 non-null  float64
 9   ebitda_USD                  37799 non-null  float64
 10  grossProfit_USD             37799 non-null  float64
 11  netIncome_USD               37799 non-null  float64
 12  cashAndCashEquivalents_USD  37799 non-null  float64
 13  shortTermInvestments_USD    377

In [4]:
df_fmp = pd.read_csv(path+'Data/mm_fmp_data_20250302_005601.csv')
df_fmp.info()

<ipython-input-4-6eb9125f07ff>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fmp = pd.read_csv(path+'Data/mm_fmp_data_20250302_005601.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181326 entries, 0 to 181325
Data columns (total 49 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   account_id                    181326 non-null  int64  
 1   account_name                  181326 non-null  object 
 2   incorporated_country          181326 non-null  object 
 3   ticker                        118117 non-null  object 
 4   Evaluation_status             180477 non-null  object 
 5   Scope_3_emissions_type        181326 non-null  object 
 6   Scope_3_emissions_amount      181326 non-null  float64
 7   Symbol_1                      181326 non-null  object 
 8   Year                          181326 non-null  int64  
 9   Market_Cap                    116071 non-null  float64
 10  Employee_Count                23567 non-null   float64
 11  ebitda                        118640 non-null  float64
 12  Revenue                       118640 non-nul

merge esg data on account id and year

In [5]:
# prompt: add data from df_fmp to df for 'environmentalScore', 'socialScore', 'governanceScore', 'ESGScore', and 'employeeCount' where 'account_id' and 'Year' match

for index, row in df_fmp.iterrows():
  df.loc[(df['account_id'] == row['account_id']) & (df['Year'] == row['Year']), 'environmentalScore'] = row['environmentalScore']
  df.loc[(df['account_id'] == row['account_id']) & (df['Year'] == row['Year']), 'socialScore'] = row['socialScore']
  df.loc[(df['account_id'] == row['account_id']) & (df['Year'] == row['Year']), 'governanceScore'] = row['governanceScore']
  df.loc[(df['account_id'] == row['account_id']) & (df['Year'] == row['Year']), 'ESGScore'] = row['ESGScore']
  df.loc[(df['account_id'] == row['account_id']) & (df['Year'] == row['Year']), 'employeeCount'] = row['employeeCount']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44079 entries, 0 to 44078
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   account_name                44079 non-null  object 
 1   account_id                  44079 non-null  int64  
 2   Year                        44079 non-null  int64  
 3   incorporated_country        44079 non-null  object 
 4   Primary activity            44079 non-null  object 
 5   Primary sector              44079 non-null  object 
 6   Market_Cap_USD              37591 non-null  float64
 7   Third_party_verification    44055 non-null  object 
 8   Revenue_USD                 37799 non-null  float64
 9   ebitda_USD                  37799 non-null  float64
 10  grossProfit_USD             37799 non-null  float64
 11  netIncome_USD               37799 non-null  float64
 12  cashAndCashEquivalents_USD  37799 non-null  float64
 13  shortTermInvestments_USD    377

merge account name and ids, symbol, and ticker

In [8]:
def remove_whitespace_punctuation(account_name):
  if account_name is None:
    return None
  account_name = re.sub(r'\s+', '', account_name)  #remove whitespace
  account_name = re.sub(r'[^\w\s]', '', account_name)  #remove punctuation
  return account_name


In [9]:
df['account_name'] = df['account_name'].apply(remove_whitespace_punctuation)

In [10]:
#displaying unique ID and account names.
try:
  unique_account_id_count = df['account_id'].nunique()
  unique_account_name_count = df['account_name'].nunique()

  print(f"Number of unique account IDs: {unique_account_id_count}")
  print(f"Number of unique account names: {unique_account_name_count}")

except Exception as e:
  print(f"An error occurred: {e}")

Number of unique account IDs: 1108
Number of unique account names: 1183


In [11]:
#group by account_id and get the most recent year for each group
df['Year'] = pd.to_datetime(df['Year'], errors='coerce').dt.year
df_recent = df.loc[df.groupby('account_id')['Year'].idxmax()]

#dictionary mapping account_id to the most recent account_name, ticker, and Symbol_1
mapping_dict = df_recent.set_index('account_id')[['account_name']].to_dict('index')

#function to replace values based on account_id
def replace_values(row):
  if row['account_id'] in mapping_dict:
    row['account_name'] = mapping_dict[row['account_id']]['account_name']
  return row

df = df.apply(replace_values, axis=1)

In [12]:
#displaying unique ID and account names.
try:
  unique_account_id_count = df['account_id'].nunique()
  unique_account_name_count = df['account_name'].nunique()

  print(f"Number of unique account IDs: {unique_account_id_count}")
  print(f"Number of unique account names: {unique_account_name_count}")

except Exception as e:
  print(f"An error occurred: {e}")

Number of unique account IDs: 1108
Number of unique account names: 1108


add random unfirom and random normal


In [14]:
df['random_normal'] = np.random.normal(loc=0, scale=1, size=len(df))
df['random_uniform'] = np.random.uniform(low=0, high=1, size=len(df))


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44079 entries, 0 to 44078
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   account_name                44079 non-null  object 
 1   account_id                  44079 non-null  int64  
 2   Year                        44079 non-null  int64  
 3   incorporated_country        44079 non-null  object 
 4   Primary activity            44079 non-null  object 
 5   Primary sector              44079 non-null  object 
 6   Market_Cap_USD              37591 non-null  float64
 7   Third_party_verification    44055 non-null  object 
 8   Revenue_USD                 37799 non-null  float64
 9   ebitda_USD                  37799 non-null  float64
 10  grossProfit_USD             37799 non-null  float64
 11  netIncome_USD               37799 non-null  float64
 12  cashAndCashEquivalents_USD  37799 non-null  float64
 13  shortTermInvestments_USD    377

output final merged df with account_id corrections and esg data to CSV

create dictionary of DFs for each scope 3 emission type

check entries per year to make sure 2022 has enough for validation

split data on 2013-2021 and 2022 validation